In [3]:
import pandas as pd
import numpy as np
from numpy.random import seed
from matplotlib import pyplot as plt
# ジュピターノートブック上でグラフを表示させるための処理
%matplotlib inline

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# ロジスティック回帰分類器

In [57]:
class LogisticRegressionMH(object):
    '''
    ロジスティック回帰分類器
    
    パラメータ
    =========
    eta : float
           学習率（0.0より大きく1.0以下の値）
    n_iter : int
            トレーニングデータのトレーニング回数
    
    属性
    =========
    w_ : 1次元配列
            適合後の重み
    errors_ : リスト
            各エポックでの誤分類数
    shuffle : bool （デフォルト : True）
            循環を回避するために各エポックでトレーニングデータをシャッフル
    random_state : int （デフォルト : None）
            シャッフルに使用するランダムステートを設定し、重みを初期化
            
    '''
    def __init__(self, eta=0.01, n_iter=10, shuffle=True, random_state=None):
        # 学習の初期化
        self.eta = eta
        
        # トレーニング回数の初期化
        self.n_iter = n_iter
        
        # 重みの初期化フラグはFalseに設定
        self.w_initialized = False
        
        # 各エポックでトレーニングデータをシャッフルするかどうかのフラグを初期化
        self.shuffle = shuffle
        
        # 引数random_stateが指定された場合は乱数種を設定
        if random_state:
            seed(random_state)
    
    def fit(self, X, y):
        '''
        トレーニングデータに適合させる
        
        パラメータ
        =========
        X : {配列のようなデータ構造}、shape = [n_samples, n_features]
                トレーニングデータ
        y : 配列のようなデータ構造、shape = [n_samples]
                目的変数
        
        戻り値
        =========
        self : object
        '''
        # 重みベクトルの生成
        self._initialize_weights(X.shape[1])
        
        # コストを格納するリストの生成
        self.cost_ = []
        
        # トレーニング回数分トレーニングデータを反復
        for i in range(self.n_iter):
            # 指定された場合はトレーニングデータをシャッフル
            if self.shuffle:
                X, y = self._shuffle(X, y)
            
            # 各サンプルのコストを格納するリストの生成
            cost = []
            
            # 各サンプルに対する計算
            for xi, target in zip(X, y):
                # 特徴量xiと目的変数yを用いた重みの更新とコストの計算
                cost.append(self._update_weights(xi, target))
                
            # サンプルの平均コストの計算
            avg_cost = sum(cost) / len(y)
            
            # 平均コストを格納
            self.cost_.append(avg_cost)
            
            return self
    
    def partial_fit(self, X, y):
        '''
        重みを再初期化することなくトレーニングデータに適合させる
        partialの意味: 部分的
        '''
        # 初期化されていない場合は初期化を実行
        if not self.w_initialized:
            self._initialize_weights(X.shape[1])
            
        # 目的変数yの要素数が2以上の場合は各サンプルの特徴量xiと目的変数targetで重みを更新
        if y.ravel().shape[0] > 1:
            for xi, target in zip(X, y):
                self._update_weights(xi, target)
                
        # 目的変数yの要素数が1の場合は、サンプル全体の特徴量Xと目的変数yで重みを更新
        else:
                self._update_weights(X, y)
                
        return self
    
    def _shuffle(self, X, y):
        '''
        トレーニングデータをシャッフル
        '''
        r = np.random.permutation(len(y))
        return X[r], y[r]
    
    def _initialize_weights(self, m):
        """
        重みを0に初期化
        """
        self.w_ = np.zeros(1 + m)
        self.w_initialized = True
        
    def _update_weights(self, xi, target):
        '''
        ロジスティック回帰の学習規則を用いて重みを更新
        target: yi
        '''
        # φ(z) = シグモイド関数: 活性化関数の出力の計算
        # output = φ(z)
        output = self.net_input(xi)
        
        # 誤差の計算
        error = (target - output)
        
        # 重み　w1, ... , wmの更新
        ## Δwj = ηΣ(yi - φ(zi)) * xij
        ## w := w + Δw
        self.w_[1:] += self.eta * xi.dot(error)
        
        # 重み　w0の更新
        self.w_[0] += self.eta * error
        
        # コストの計算 
        ## J(w) = - Σ { yi * log(φ(zi)) + (1 - yi)*log(1 - φ(zi)) }
        cost = - (target * np.log(output) + (1 - target) * np.log(1 - output))
        
        return cost
    
    def net_input(self, X):
        '''
        総入力を計算
        '''
        return np.dot(X, self.w_[1:] + self.w_[0])
  
    def sigmoid(z):
        '''
        シグモイド関数
        '''
        return 1.0 / (1.0 + np.exp(-z))

    def activation(self, X):
        '''
        線形活性化関数の出力を計算
        '''
        z = self.net_input(X)
        return sigmoid(z)
    
    def predict_proba(self, X):
        return self.activation(X)
        
    def predict(self, X):
        '''
        1ステップ後のクラスラベルを返す
        '''
        return np.where(self.activation(X) >= 0.0, 1, -1)

## Irisデータセットの取得と整形

In [46]:
# Irisデータセットをロード
iris = datasets.load_iris()

# 3, 4列目の特徴量を抽出
X = iris.data[:, [2, 3]]

# クラスラベルを取得
y = iris.target

# トレーニングデータとテストデータに分割
## 全体の30%をテストデータにする
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Irisデータセットを標準化
sc = StandardScaler()

# トレーニングデータの平均と標準偏差を計算
sc.fit(X_train)

# 平均と標準偏差を用いて標準化
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## ロジスティック回帰分類器にデータを適用

In [47]:
lg_mh = LogisticRegressionMH(n_iter=15, eta=0.01, random_state=1)

In [48]:
lg_mh.fit(X_train_std, y_train)

In [49]:
lg_mh.cost_

[2.1681139006661487]

In [53]:
from sklearn.linear_model import LogisticRegression

# ロジスティック回帰のインスタンスを生成
lr = LogisticRegression(random_state=1)

# トレーニングデータをモデルに適合させる
lr.fit(X_train_std, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [55]:
lr.predict(X_test_std)

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 2, 1, 1, 2, 0, 2, 0,
       0])

In [56]:
lg_mh.predict(X_test_std)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1])